# Build a Python Windows app with your smartphone
> "Build a Python Windows UI application using a Gitlab Windows runner."

- toc: false
- branch: master
- badges: true
- comments: true
- author: Jaco Verster
- categories: [python, tkinter, CI/CD]

Part of my work involves building a Windows apps that run powerful Python scripts for my users. I typically use [tkinter](https://docs.python.org/3/library/tkinter.html) to build the UI and I've been playing with the idea to automate the compilation step by using a CI/CD runner. I recently found out that you can spin up a Windows shared runner on Gitlab for free so I just had to test it out. Now the catch was that I wasn't close to my computer at the time but I had some time to kill. I had my phone with me so I thought let me see if I can build the whole thing using only my smartphone. I just want to build a very simple test app, so I can copy-paste code from the internet - it should be doable without my IDE right?

So I set  out to build this using only my smartphone!

I created a new project on Gitlab and created the `.gitlab-ci.yml` file first. I copied the code from [this](https://stackoverflow.com/a/67414819/16190875) stackoverflow answer and modified it. I first tried editing it using the Gitlab web editor but that proved impossible - the web editor is not optimised for mobile devices so it was impossible to move around. I decided to us a document editor on my smartphone (Google Docs) instead. I copied the code there, made my changes and then copied it to Gitlab using the Gitlab web editor.

The final `.gitlab-ci.yml` code spins up the shared Windows runner, installs Python 3.9 using choco, then installs pyinstaller and builds the app and saves the output dist/app.exe to a build artifact that can later be downloaded.

In [ ]:
# .gitlab-ci.yml

.shared_windows_runners:
   tags:
   - shared-windows
   - windows
   - windows-1809

 build:
   extends:
   - .shared_windows_runners
   stage: build
   script:
   - choco install python --version=3.9.4 -y -f
   - "C:\\Python39\\python.exe -m pip install pyinstaller"
   - "C:\\Python39\\Scripts\\pyinstaller.exe app.py -F"
   artifacts:
     paths:
       - dist/app.exe


Next I created `app.py` for the Python source code. I simply copied some code from this Real Python Tutorial on [Python GUI Programming With Tkinter](https://realpython.com/python-gui-tkinter/#building-a-text-editor-example-app) without modifying it. It is a text editor application that can create, open, edit, and save text files, a simple example to test the idea. The code looks like this:

In [ ]:
# app.py

import tkinter as tk
from tkinter.filedialog import askopenfilename, asksaveasfilename

def open_file():
    """Open a file for editing."""
    filepath = askopenfilename(
        filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
    )
    if not filepath:
        return
    txt_edit.delete("1.0", tk.END)
    with open(filepath, mode="r", encoding="utf-8") as input_file:
        text = input_file.read()
        txt_edit.insert(tk.END, text)
    window.title(f"Simple Text Editor - {filepath}")

def save_file():
    """Save the current file as a new file."""
    filepath = asksaveasfilename(
        defaultextension=".txt",
        filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")],
    )
    if not filepath:
        return
    with open(filepath, mode="w", encoding="utf-8") as output_file:
        text = txt_edit.get("1.0", tk.END)
        output_file.write(text)
    window.title(f"Simple Text Editor - {filepath}")

window = tk.Tk()
window.title("Simple Text Editor")

window.rowconfigure(0, minsize=800, weight=1)
window.columnconfigure(1, minsize=800, weight=1)

txt_edit = tk.Text(window)
frm_buttons = tk.Frame(window, relief=tk.RAISED, bd=2)
btn_open = tk.Button(frm_buttons, text="Open", command=open_file)
btn_save = tk.Button(frm_buttons, text="Save As...", command=save_file)

btn_open.grid(row=0, column=0, sticky="ew", padx=5, pady=5)
btn_save.grid(row=1, column=0, sticky="ew", padx=5)

frm_buttons.grid(row=0, column=0, sticky="ns")
txt_edit.grid(row=0, column=1, sticky="nsew")

window.mainloop()

After I committed the changes to Gitlab the runner spun up and built the app and created a downloadable app.exe artifact. I later downloaded the app and to test it on my Windows machine and it ran without any issues. I showed that it is possible to build a Windows Python app without a Windows computer using a free Gitlab Windows runner (and a smartphone in this case!). Next steps will be to try and automate app deployment using the built artifact.

Follow the links to find out more about [Windows Development](https://docs.gitlab.com/ee/development/windows.html) using Gitlab or look at the [pre-installed software](https://gitlab.com/gitlab-org/ci-cd/shared-runners/images/gcp/windows-containers/blob/main/cookbooks/preinstalled-software/README.md) on their Windows runners. 